In [1]:
import pandas as pd
import warnings
from functools import reduce

# Framework d'Analyse de Portefeuille

## Import & Traitement des Données

In [2]:
class DataFile:
    def __init__(self, id: str, filepath: str, filename: str, sheet: bool, file_format: str, select_col: list, name_col: list, first_date : str):
        self.id = id
        self.filepath = filepath
        self.filename = filename
        self.sheet = sheet
        self.file_format = file_format
        self.select_col = select_col
        self.name_col = name_col
        self.first_date = first_date
        self.df = self.load_data()

    def load_data(self):
        """Cette méthode charge le fichier sélectionné dans un dataframe. Pour ce faire, elle fait appel à trois autres méthodes : import_data(), filter_columns() et clean_data().Elle retourne donc un dataframe, enregistré dans self.df."""
        
        data = self.import_data()
        data_filtered = self.filter_columns(data)
        final_data = self.clean_data(data_filtered)
        return final_data

    def import_data(self):
        """Cette méthode se charge d'importer les données depuis les fichiers. Elle tient compte du format du fichier (excel ou csv, plusieurs feuilles ou pas) renseigné en input. Elle renvoit donc un dataframe."""
        
        dict_format = {"xlsx":"excel",
                       "csv":"csv"}   
        if self.file_format in dict_format.keys():
            if self.sheet: # si le excel contient plusieurs feuilles (self.sheet = True), selectionne la bonne.
                data = getattr(pd,f"read_{dict_format[self.file_format]}")(f"{self.filepath}/{self.filename}.{self.file_format}",sheet_name = self.id)
            else:
                data = getattr(pd,f"read_{dict_format[self.file_format]}")(f"{self.filepath}/{self.filename}.{self.file_format}")            
            return data
        else:
            raise ValueError("Unsupported file type. Use 'xlsx' or 'csv'.")
    
    def filter_columns(self, data):
        """Cette méthode filtre les colonnes qui nous interessent dans le cadre du projet (dates, VL pour les fonds, données US et Monde pour les facteurs AQR ...) et renvoie le dataframe filtré. """
        
        data = data[[data.columns[idx] for idx in self.select_col]] 
        data.columns = self.name_col # on renomme les colonnes avec les noms renseignés en input
        return data
    
    def clean_data(self, data_filtered):
        """Cette méthode s'assure de la propreté des données en enlevant les informations non nécessaire au projet et s'assurant du bon format des données."""
        
        while data_filtered.iloc[0,0] != self.first_date: # tant que la première cellule du dataframe ne correspond pas à la première date de la série
            data_filtered = data_filtered.iloc[1:] # on supprime la première ligne
        
        data_filtered.Date = pd.to_datetime(data_filtered.Date, infer_datetime_format=True) # format datetime pour les dates
        filter_float = data_filtered.columns.difference(['Date'])
        data_filtered[filter_float] = data_filtered[filter_float].replace(",",".",regex=True).astype(float) # format float pour le reste

        data_filtered = data_filtered.dropna(axis="index") # on supprime les lignes sans données
        return data_filtered

### Fonds & Valeurs Liquidatives

#### Fonds 1 : JPM America Equity C (Acc)

In [3]:
jpm_vl = DataFile(
    id="LU0129459060",
    filepath="funds",
    filename= "JPMorgan Funds - America Equity Fund",
    sheet = False,
    file_format= "xlsx",
    select_col = [0,1],
    name_col = ['Date','LU0129459060'],
    first_date= "30.08.2019",
            )

#### Fonds 2 : Schroder International Selection Fund Global Sustainable Growth C Acc

In [5]:
schroder_vl = DataFile(
    id="LU0557290854",
    filepath="funds",
    filename= "Schroder International Selection Fund Global Sustainable Growth C Accumulation USD",
    sheet = False,
    file_format= "csv",
    select_col = [4,5],
    name_col = ['Date','LU0557290854'],
    first_date= "09.10.2024",
            )

warnings.filterwarnings('ignore')

#### Fonds 3 : AQR Large Cap Multi-Style Fund

In [6]:
aqr_vl = DataFile(
    id="US00203H4956",
    filepath="funds",
    filename= "AQR Large Cap Multi-Style Fund Daily Price History",
    sheet = False,
    file_format= "csv",
    select_col = [2,6],
    name_col = ['Date','US00203H4956'],
    first_date= "26/03/2013 00:00",
            )

#### Tableau des valeurs liquidatives de nos trois fonds du 30/08/2019 au 30/08/2024

In [7]:
funds_vl = pd.merge(pd.merge(jpm_vl.df, schroder_vl.df, on="Date", how="inner"), aqr_vl.df, on="Date", how="inner")
funds_vl

,Date,LU0129459060,LU0557290854,US00203H4956
0,2019-08-30,35.05,218.0506,16.42
1,2019-09-04,34.97,214.5538,16.46
2,2019-09-05,35.47,212.1062,16.72
3,2019-09-09,35.41,222.6949,16.75
4,2019-09-10,35.16,218.6702,16.77
...,...,...,...,...
1084,2024-08-26,77.20,435.9759,20.39
1085,2024-08-27,76.69,434.1793,20.40
1086,2024-08-28,76.83,436.5764,20.33
1087,2024-08-29,76.90,437.2779,20.33


### Facteurs de performance selon AQR

In [8]:
factors_list = ["MKT","SMB","HML FF","HML Devil","UMD","RF"]
factors = {}

for factor in factors_list:
    factors[factor] = DataFile(
        id= factor,
        filepath= "aqr factors",
        filename= "Betting Against Beta Equity Factors Daily",
        sheet= True,
        file_format= "xlsx",
        select_col= [0,25,26,27] if factor!="RF" else [0,1],
        name_col= ["Date",f"{factor} USA",f"{factor} Global",f"{factor} Global Ex USA"] if factor!="RF" else ["Date","Risk Free Rate"],
        first_date= "01/03/1927"
        )

#### Tableau des facteurs AQR (US, Globaux, et Globaux sans US) du 07/01/1988 au 31/10/2024 (fréquence journalière)

In [ ]:
dataframes = [f.df for f in factors.values()]

# On consolide les dataframe de chaque facteur de performance en un mega dataframe
aqr_factors = reduce(
    lambda left, right: pd.merge(left, right, on="Date", how="inner"),
    dataframes
)

aqr_factors

,Date,MKT USA,MKT Global,MKT Global Ex USA,SMB USA,SMB Global,SMB Global Ex USA,HML FF USA,HML FF Global,HML FF Global Ex USA,HML Devil USA,HML Devil Global,HML Devil Global Ex USA,UMD USA,UMD Global,UMD Global Ex USA,Risk Free Rate
0,1988-07-01,-0.006224,-0.007512,0.002349,0.002916,0.002552,0.004013,0.000507,-0.000110,-0.004913,-0.000932,-0.002641,-0.005892,-0.003993,-0.005824,0.000784,0.000262
1,1988-07-04,-0.008430,-0.012907,-0.005941,0.000506,0.000836,-0.000891,-0.002626,-0.004331,0.015692,-0.003423,-0.005651,0.009979,-0.002074,-0.003214,-0.002052,0.000262
2,1988-07-05,0.012208,0.012964,0.011222,-0.008576,-0.006775,-0.006581,0.002556,0.004686,0.017758,0.002392,0.003996,0.016071,0.007318,0.011670,-0.002165,0.000262
3,1988-07-06,0.002862,0.010396,0.009918,-0.004814,-0.015155,-0.002306,0.006114,0.009240,0.014669,0.002638,0.003531,0.007472,0.004378,0.005867,-0.006123,0.000262
4,1988-07-07,0.006536,0.010365,-0.000900,0.000632,0.000886,0.008475,0.009954,0.016454,0.002779,0.008370,0.014065,-0.001282,-0.001012,-0.001178,-0.000439,0.000262
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9475,2024-10-25,-0.001176,-0.000581,0.001680,0.000468,-0.000764,-0.001684,-0.003333,0.001956,0.004634,-0.001852,0.002529,0.005336,-0.003779,-0.003307,-0.001591,0.000181
9476,2024-10-28,0.003668,0.003367,0.003945,0.005159,-0.001771,-0.002058,-0.000081,-0.002438,-0.001839,0.001775,-0.000529,-0.000102,-0.001022,-0.001973,0.000505,0.000181
9477,2024-10-29,-0.000434,-0.002743,-0.007043,0.001329,0.002909,0.003680,-0.006190,-0.003211,-0.002471,-0.008773,-0.004688,-0.003103,0.004090,0.001134,0.000934,0.000181
9478,2024-10-30,-0.003724,-0.003621,-0.006081,0.000686,0.003481,0.006245,0.003448,0.000322,-0.000373,0.002725,-0.000582,0.000131,0.001012,0.003905,0.006664,0.000181
